In [2]:
# -*- coding: utf-8 -*-

from gensim import corpora, matutils
import pandas as pd
import MeCab
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer


def readCSV(file_pass, name, file_type):
    df = pd.read_csv(file_pass+name+file_type, dtype={'label': 'str'}, encoding="utf-8")
    return df


def sep(body):
    mt = MeCab.Tagger('mecabrc')
    mt.parse("")
    node = mt.parseToNode(str(body))
    return getNouns(node)


def getNouns(node):
    words = []
    word_stack = ""
    while node:
        noun = node.feature.split(",")
        word = node.surface
        if(checkNoun(noun)):
            word_stack += word
        if(word_stack != "" and noun[0] != '名詞'):
            if(checkNum(word_stack)):
                words.append(word_stack)
            word_stack = ""
        node = node.next
#     words = " ".join(words)
    return words


# 名詞のみを抽出
def checkNoun(noun):
    if(noun[0] == '名詞'):
        if(noun[1] != '副詞可能'):
            if(noun[1] != '非自立'):
                if(noun[1] != '代名詞'):
                    if(noun[1] != '助詞類接続'):
                        return True
                    else:
                        return False
                else:
                    return False
            else:
                return False
        else:
            return False
    else:
        return False


# 12月 1.2%などの数字関連を省く
def checkNum(word_stack):
    mt = MeCab.Tagger()
    mt.parse("")

    node = mt.parseToNode(word_stack)
    while node:
        noun = node.feature.split(",")
        if(noun[1] == '数'):
            return False
        node = node.next

    return True

#ラベル付け
def ch(cat_list):
    cats = []
    for cat, i in zip(cat_list, range(len(cat_list))):
        if(cat == "1"):
            cats.append(i)
    if(cats == []):
        #すべての要素が0の時にその他のラベルである13をあてる
        cats.append(13)
    return np.asarray(cats)


Using TensorFlow backend.


In [3]:
file_pass = "./"
#3が自作関数での分割
file_name = "reuters_article"
file_type = ".csv"
dict_list = []
# print(dict_list)
df = readCSV(file_pass, file_name, file_type)
# print(df["label"])
body = []
label_train = []
# bodyに

In [4]:
body = [sep(row) for row in df["body"]]

In [5]:
label_train = []
for i in df["label"]:
    label_train.append(ch(i))

In [6]:
label_train = MultiLabelBinarizer().fit_transform(label_train)

In [7]:
dictionary = corpora.Dictionary(body)
dictionary.filter_extremes(no_below=20, no_above=0.4)

In [8]:
data_train = []
# print(body)
for b in body:
    tmp = dictionary.doc2bow(b)
    data_train.append(list(matutils.corpus2dense([tmp], num_terms=len(dictionary)).T[0]))
    # print(dense)

In [9]:
# len(label_train)
data_train_s, data_test_s, label_train_s, label_test_s = train_test_split(data_train, label_train, test_size=0.4)

# len(data_train_s)

estimator = RandomForestClassifier(n_estimators=15, random_state=0)
estimator.fit(data_train_s, label_train_s)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=15, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [10]:
print(estimator.score(data_test_s, label_test_s))

0.327596996245


In [11]:
y_train_pred = estimator.predict(data_train_s)
y_test_pred = estimator.predict(data_test_s)

In [13]:
from sklearn.metrics import precision_score, recall_score, f1_score

In [14]:
print(precision_score(label_test_s, y_test_pred, average='micro'))
print(recall_score(label_test_s, y_test_pred, average='micro'))
print(f1_score(label_test_s, y_test_pred, average='micro'))
print('-'*40)
print(precision_score(label_test_s, y_test_pred, average='macro'))
print(recall_score(label_test_s, y_test_pred, average='macro'))
print(f1_score(label_test_s, y_test_pred, average='macro'))

0.804358295208
0.417301686446
0.549514722816
----------------------------------------
0.768617255674
0.175236479675
0.237569703282


/Users/hurry/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/hurry/.pyenv/versions/anaconda3-4.0.0/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
def sum(data):
    total = [0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    for row in data:
        for r, i in zip(row, range(len(row))):
            total[i] += int(r)
            
    return total

In [ ]:
pred = sum(y_test_pred)
true = sum(label_test_s)

In [ ]:
print(pred)
print(true)